# SVM segmentation from VISUAL information only

In [1]:
from skimage import feature
from sklearn.svm import SVC
import skimage as si
from skimage.io import imread, imshow
from skimage.transform import resize
from sklearn.metrics import f1_score
from skimage.feature import multiscale_basic_features, canny
from skimage.filters import threshold_otsu
from skimage.exposure import adjust_log
from glob import glob
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
imgs = glob("../Data/N2DH-GOWT1/img/*.tif")
masks = glob("../Data/N2DH-GOWT1/gt/tif/*")
IMG_SIZE = 250
print(f"Using {IMG_SIZE} of image size")
print(f"We have {len(imgs)} with {len(masks)} masks")

## Using pixel intensity only

This is a first approach to solve this, in this case we will generate a features matrix with the shape (Number_of_pixels, 1), since the original images are too big to do quick testing, I suggest to resize them, you can use the parameter IMG_SIZE to do just that.

$$ X = N_{pixels} \times 1$$
$$W = 1 \times 1$$
$$Y = N_{pixels} \times 1$$

Remember that $N_{pixels}$ is the total amount of pixels that are in the training data, in this case two images of IMG_SIZE, $$N_{pixels} = \operatorname{N of images}\times(\operatorname{IMG\_SIZE}^2)$$

In [ ]:
def compareResults(prediction, image_to_test=4):
    """
    Helper funtion to visualize results
    """
    fig, ax = plt.subplots(ncols=3, dpi=144)
    _imgs = imread(imgs[image_to_test])
    _imgs = resize(_imgs, (IMG_SIZE, IMG_SIZE))
    _mask = imread(masks[image_to_test])
    _mask = resize(_mask, (IMG_SIZE, IMG_SIZE))
    ax[0].imshow(_imgs, cmap='gray')
    ax[1].imshow(prediction.reshape((IMG_SIZE, IMG_SIZE)), cmap='gray')
    ax[2].imshow(_mask, cmap='gray')
    for ax, label in zip(ax, ["Original Image", "Prediction", "Ground Truth"]):
        ax.axis('Off')
        ax.set_title(label)
    fig.tight_layout()

In [ ]:
print(f"NPixels used for training {IMG_SIZE*IMG_SIZE*2}")

In [ ]:
def features_pixelValue(img, mask):
    """
    Generate features using the pixel value
    """
    img_array = imread(img, as_gray=True)
    mask_array = imread(mask, as_gray=True)
    # Binarize mask, everything that is bigger than zero is one
    # Resize images to reduce computing costs, notice this process makes image values between 0 and 1
    img_array = resize(img_array, (IMG_SIZE, IMG_SIZE))
    mask_array = resize(mask_array, (IMG_SIZE, IMG_SIZE), preserve_range=True)
    mask_array = np.clip(mask_array, 0, 1).round()

    np.unique(mask_array, return_counts=True)

    # Reshaping so every pixel is a row
    features = img_array.reshape(-1, 1)
    mask_array = mask_array.reshape(-1, 1)
    return features, mask_array

Here I am training the SVM version from scikit-learn, it's up to you to adapt the features matrix to your version. We are training on the first two images

In [ ]:
all_imgs, all_masks = [], []
for img, mask in zip(imgs[:2], masks[:2]):
    img_svc, mask_svc = features_pixelValue(img, mask)

    # Collecting
    all_imgs.append(img_svc)
    all_masks.append(mask_svc)

# Lets make a matrix
imgs_mat = np.vstack(all_imgs)
masks_mat = np.vstack(all_masks)

svc_pv = SVC(kernel='linear', C=20)
svc_pv.fit(imgs_mat, masks_mat)
print("Done Training")

In [ ]:
image_to_test = 4
im_svc, mask_svc = features_pixelValue(imgs[image_to_test], masks[image_to_test])
pred_pv = svc_pv.predict(im_svc)
f1 = round(f1_score(mask_svc, pred_pv), 3)
print(f"F1/Dice Score: {f1}")

compareResults(pred_pv, image_to_test)

As you can see, this is really slow for large images and the black areas inside a cell are not correctly segmented which is normal since we are only using the pixel intensity value. We have zero knowledge of the surroundings.
To improve this we can add more features, let's try adding edge information.
$$ X = N_{pixels} \times 2$$
$$W = 1 \times 2$$
$$Y = N_{pixels} \times 1$$

## Adding Edge information

In [ ]:
def features_pixelValue_edge(img, mask):
    """This function will process the image and the image ground truth (AKA mask), the mask will be just resized.
        We will generate features for the SVM in order to make predictions,
       The first column of the features will be the pixel intensity between 0 to 1
       The second column of the features will be the canny filter for edge detection.

    """
    img_array = imread(img, as_gray=True)
    mask_array = imread(mask, as_gray=True)


    img_array = resize(img_array, (IMG_SIZE, IMG_SIZE))
    edges = canny(img_array, sigma=.5)
    img_array = img_array.reshape(-1, 1)
    edges = edges.reshape(-1, 1)
    features = np.hstack([img_array, edges])

    mask_array = resize(mask_array, (IMG_SIZE, IMG_SIZE), preserve_range=True)
    mask_array = np.clip(mask_array, 0, 1).round()
    mask_array = mask_array.reshape(-1)
    return features, mask_array

In [ ]:
all_imgs, all_masks = [], []
N_Train_images_used = 2
for img, mask in zip(imgs[:N_Train_images_used], masks[:N_Train_images_used]):
    img_svc, mask_svc = features_pixelValue_edge(img, mask)

    # Collecting
    all_imgs.append(img_svc)
    all_masks.append(mask_svc)

# Lets make a matrix
imgs_mat = np.vstack(all_imgs)
masks_mat = np.vstack(all_masks)
print("Training .....", end="")
svc_pv_edge = SVC(kernel='linear', C=20)
svc_pv_edge.fit(imgs_mat, masks_mat.reshape(-1))
print(" Done :)")

In [ ]:
image_to_test = 5
im_svc, mask_svc = features_pixelValue_edge(imgs[image_to_test], masks[image_to_test])
pred_pv_edge = svc_pv_edge.predict(im_svc)
f1 = np.round(f1_score(mask_svc, pred_pv_edge), 3)
print(f"F1/Dice Score: {f1}")

compareResults(pred_pv_edge, image_to_test)

Seems it did nothing much, maybe if we improve the image contrast, let's use OTSU's thresholding

## Thresholding the image

In [ ]:
def features_pixelValue_otsu(img, mask):
    """
    Generate features using the pixel value after using the Otsu's threshold
    """
    img_array = imread(img, as_gray=True)
    thr = threshold_otsu(img_array)
    img_array = (img_array > thr).astype(float)
    mask_array = imread(mask, as_gray=True)
    # Binarize mask, everything that is bigger than zero is one
    # Resize images to reduce computing costs, notice this process makes image values between 0 and 1
    img_array = resize(img_array, (IMG_SIZE, IMG_SIZE))
    mask_array = resize(mask_array, (IMG_SIZE, IMG_SIZE), preserve_range=True)
    mask_array = np.clip(mask_array, 0, 1).round()

    np.unique(mask_array, return_counts=True)

    # Reshaping so every pixel is a row
    features = img_array.reshape(-1, 1)
    mask_array = mask_array.reshape(-1)
    return features, mask_array

In [ ]:
all_imgs, all_masks = [], []
N_Train_images_used = 2
for img, mask in zip(imgs[:N_Train_images_used], masks[:N_Train_images_used]):
    img_svc, mask_svc = features_pixelValue_otsu(img, mask)

    # Collecting
    all_imgs.append(img_svc)
    all_masks.append(mask_svc)

# Lets make a matrix
imgs_mat = np.vstack(all_imgs)
masks_mat = np.vstack(all_masks)
print("Training .....", end="")
svc_otsu_pv = SVC(kernel='linear', C=20)
svc_otsu_pv.fit(imgs_mat, masks_mat.reshape(-1))
print("Done :)")

In [ ]:
image_to_test = 5
im_svc, mask_svc = features_pixelValue_otsu(imgs[image_to_test], masks[image_to_test])
pred_otsu_pv = svc_otsu_pv.predict(im_svc)
f1 = round(f1_score(mask_svc, pred_otsu_pv), 3)
print(f"F1/Dice Score: {f1}")

compareResults(pred_otsu_pv, image_to_test)

Take into the account that the training is done using a smaller image and just two of the for training. Also the results are compared with just one image, that probably not enough to draw a proper conclusion. Let's use something more powerful.

## Adding more features using more information

In [ ]:
def features_multiscale(img, mask):
    img_array = imread(img, as_gray=True)
    mask_array = imread(mask, as_gray=True)
    # Binarize mask, everything that is bigger than zero is one
    # Resize images to reduce computing costs, notice this process makes image values between 0 and 1
    img_array = resize(img_array, (IMG_SIZE, IMG_SIZE))
    mask_array = resize(mask_array, (IMG_SIZE, IMG_SIZE), preserve_range=True)
    mask_array = np.clip(mask_array, 0, 1).round()

    features = multiscale_basic_features(img_array,
                                         intensity=True,
                                         edges=True,
                                         texture=True,
                                         sigma_min=1,
                                         sigma_max=16)
    mask_array = mask_array.reshape(-1, 1)
    features = features.reshape(-1, features.shape[-1])
    return features, mask_array

In [ ]:
all_imgs, all_masks = [], []
N_Train_images_used = 2
for img, mask in zip(imgs[:N_Train_images_used], masks[:N_Train_images_used]):
    img_svc, mask_svc = features_multiscale(img, mask)

    # Collecting
    all_imgs.append(img_svc)
    all_masks.append(mask_svc)

# Lets make a matrix
imgs_mat = np.vstack(all_imgs)
masks_mat = np.vstack(all_masks)
print("Training .....", end="")
svc_multiscale = SVC(kernel='linear', C=20)
svc_multiscale.fit(imgs_mat, masks_mat.reshape(-1))
print("Done :)")

For the sake of clarity the feature matrix looks like, remember, this matrix is generated from the image of the cells.

In [ ]:
import pandas as pd
df = pd.DataFrame(imgs_mat)
df.index.name = "PixelNum"
df.columns = [f"feature_{i}" for i in range(df.shape[1])]
df.head(5)

In [ ]:
image_to_test = 5
im_svc, mask_svc = features_multiscale(imgs[image_to_test], masks[image_to_test])
pred_multiscale = svc_multiscale.predict(im_svc)
f1 = round(f1_score(mask_svc, pred_multiscale), 3)
print(f"F1/Dice Score: {f1}")

compareResults(pred_multiscale, image_to_test)

You should also play with increasing the contrast, sharpening the image.
